## SOLANA onchain bot
## Plain agentic usage of Assisterr SLM models 

In [38]:
!pip install partial_json_parser
!pip install termcolor
!pip install json-repair

In [30]:
import requests
import json
from partial_json_parser import loads
from json_repair import repair_json
from termcolor import colored


In [3]:
# API Key can be obtained from https://build.assisterr.ai/ailab 
ASSISTERR_API_KEY = ''
ASSISTERR_BASE_URL = 'https://api.assisterr.ai'

In [23]:
# Basic ASSISTERR SLM class to make calls and construct prompt with history context

class ASSISTERR_SLM:
    def __init__(self, model_slug):
        self.model = model_slug

    def construct_prompt(self, question, chat_history):
        """Create prompt
        Args:
            question: str; query from user
            chat_history: list; list of formatted conversation history
        Returns:
            Prompt text to be used in the model
        """

        chat_history_formatted = self.format_chat_history(chat_history)
        msg = f"<|start_header_id|>user<|end_header_id|>\n{question}<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n"
        final_prompt = chat_history_formatted + msg
        return final_prompt

    def format_chat_history(self, chat_history):
        """Format chat history
        Args:
            chat_history: list; list of conversation history
        Returns:
            Formatted chat history string
        """
        formatted_history = ""
        for entry in chat_history:
            user_input = entry['user']
            assistant_response = entry['assistant']
            formatted_history += f"<|start_header_id|>user<|end_header_id|>\n{user_input}<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n{assistant_response}<|eot_id|>\n"
        return formatted_history

    def get_response(self, question):
        """Generate response using
        Args:
            question: str; question from the user
        Returns:
            The generated AI response text
        """

        prompt = self.construct_prompt(question, chat_history_simulation)

        request = json.dumps(prompt)

                
        url = f'{ASSISTERR_BASE_URL}/api/v1/slm/{self.model}/chat/'
    
        response = requests.post(url=url, headers={'X-Api-Key': ASSISTERR_API_KEY}, json={'query': request})
        result = response.json()['message']
        
        chat_history_simulation.append({'user': question, 'assistant': result})
        return result



In [36]:
# SOLANA API action calling based on Assisterr SLM chosen endpoint and provided payload to make call with

def make_call(payload):
    url = "https://api.mainnet-beta.solana.com"
    headers = {
        "Content-Type": "application/json"
    }

    payload.update({"jsonrpc": "2.0", "id": 1})
    response = requests.post(url, headers=headers, json=payload)

    return response.json()

In [37]:
# Some chat history simulation to control agent context
chat_history_simulation = [
    {"user": "",
     "assistant": "Hey I'm here to help! what are you interested in?"},
]

# Initializae slm with already created model solana_api_bot
agent = ASSISTERR_SLM('solana_api_bot')


# some chat simulation to perform onchain operations
# 1. Grab user input
# 2. Call SLM
# 3. Automatically call SOLANA API based on SLM response

while True:
    query = input()
    api_call = loads(repair_json(agent.get_response(query)))
    print(colored(api_call['responding_message'], 'red'))
    call_payload = api_call['payload']
    while call_payload:
        result = make_call(api_call['payload'])
        response = loads(repair_json(agent.get_response({'payload': result})))
        print(colored(response['responding_message'], 'red'))

        if response['payload']:
            print(response['payload'])
            call_payload = response['payload']
        else:
            call_payload = None    

 what is the balance of benRLpbWCL8P8t51ufYt522419hGF5zif3CqgWGbEUm wallet


I'm a Solana chain API bot. You can execute any onchain operations for users based on their requests.
The balance of the benRLpbWCL8P8t51ufYt522419hGF5zif3CqgWGbEUm wallet is 1941.503393146 SOL.


KeyboardInterrupt: Interrupted by user